In [1]:
pip install sqlalchemy mysqlclient pandas

     ---------------------------------------- 2.1/2.1 MB 4.3 MB/s eta 0:00:00
     ------------------------------------- 203.2/203.2 kB 12.9 MB/s eta 0:00:00
     -------------------------------------- 292.8/292.8 kB 9.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
from sqlalchemy import create_engine, Column, Integer, String, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import pandas as pd

# Crear la conexión a la base de datos
user = 'root'
password = 'Chowder1226'
host = 'localhost'
port = '3308'
database = 'dataset'

engine = create_engine(f'mysql+mysqldb://{user}:{password}@{host}:{port}/{database}')
Base = declarative_base()

# Definir la estructura de la tabla
class Municipio(Base):
    __tablename__ = 'municipios'
    CodigoMunicipio = Column(Integer, primary_key=True)
    NombreMunicipio = Column(String(100))
    Edad = Column(Integer)
    Hombres_Cabecera = Column(Integer)
    Mujeres_Cabecera = Column(Integer)
    Hombres_CentroPoblado = Column(Integer)
    Mujeres_CentroPoblado = Column(Integer)
    HombresRuralDisperso = Column(Integer)
    MujeresRuralDisperso = Column(Integer)

# Crear la tabla en la base de datos
Base.metadata.create_all(engine)

# Crear una sesión
Session = sessionmaker(bind=engine)
session = Session()


C:\Users\user\AppData\Local\Temp\ipykernel_12116\3174049991.py:14: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
dataset = r"D:\Data-Science\Poblaci_n_Antioquia_censada_2018_por_tipo_territorio__sexo_y_edades_simples_20240330.csv"

In [17]:
# Leer el dataset
datos = pd.read_csv(dataset, delimiter=';', encoding='latin1')

# Subir los datos a la base de datos
datos.to_sql('municipios', con=engine, if_exists='replace', index=False)

12825

In [18]:
query = text("SELECT * FROM municipios WHERE Edad > 30")
with engine.connect() as conn:
    result = conn.execute(query)
    
    
    df_result = pd.DataFrame(result.fetchall(), columns=result.keys())
    print(df_result)



      CodigoMunicipio NombreMunicipio  Edad  Hombres_Cabecera  \
0                5001        Medellín    31             19406   
1                5001        Medellín    32             18874   
2                5001        Medellín    33             18628   
3                5001        Medellín    34             17601   
4                5001        Medellín    35             17940   
...               ...             ...   ...               ...   
8946             5895        Zaragoza    94                 0   
8947             5895        Zaragoza    97                 1   
8948             5895        Zaragoza    98                 1   
8949             5895        Zaragoza   102                 0   
8950             5895        Zaragoza   104                 0   

      Mujeres_Cabecera  Hombres_CentroPoblado  Mujeres_CentroPoblado  \
0                19308                    153                    169   
1                19525                    165                    156   
2  

In [19]:
query = text("""
SELECT NombreMunicipio, 
       SUM(Hombres_Cabecera + Mujeres_Cabecera + Hombres_CentroPoblado + Mujeres_CentroPoblado + HombresRuralDisperso + MujeresRuralDisperso) as Total_Poblacion
FROM municipios
GROUP BY NombreMunicipio
""")
with engine.connect() as conn:
    result = conn.execute(query)

    
    df_result = pd.DataFrame(result.fetchall(), columns=result.keys())
    print(df_result)

    NombreMunicipio Total_Poblacion
0         Abejorral           17599
1          Abriaquí            2159
2        Alejandría            3839
3             Amagá           26821
4            Amalfi           20265
..              ...             ...
120            Yalí            6234
121         Yarumal           36337
122         Yolombó           19010
123           Yondó           17597
124        Zaragoza           24067

[125 rows x 2 columns]


In [20]:
query = text("""
SELECT NombreMunicipio, 
       SUM(Hombres_Cabecera + Mujeres_Cabecera) as Poblacion_Cabecera
FROM municipios
GROUP BY NombreMunicipio
""")
with engine.connect() as conn:
    result = conn.execute(query)

    
    df_result = pd.DataFrame(result.fetchall(), columns=result.keys())
    print(df_result)


    NombreMunicipio Poblacion_Cabecera
0         Abejorral               6957
1          Abriaquí                708
2        Alejandría               2215
3             Amagá              13032
4            Amalfi              11685
..              ...                ...
120            Yalí               3048
121         Yarumal              26783
122         Yolombó               6175
123           Yondó               9515
124        Zaragoza              11095

[125 rows x 2 columns]


In [21]:
query = text("""
SELECT NombreMunicipio, 
       SUM(Hombres_CentroPoblado + Mujeres_CentroPoblado + HombresRuralDisperso + MujeresRuralDisperso) as Poblacion_CentroPoblado_Rural
FROM municipios
GROUP BY NombreMunicipio
""")
with engine.connect() as conn:
    result = conn.execute(query)

   
    df_result = pd.DataFrame(result.fetchall(), columns=result.keys())
    print(df_result)


    NombreMunicipio Poblacion_CentroPoblado_Rural
0         Abejorral                         10642
1          Abriaquí                          1451
2        Alejandría                          1624
3             Amagá                         13789
4            Amalfi                          8580
..              ...                           ...
120            Yalí                          3186
121         Yarumal                          9554
122         Yolombó                         12835
123           Yondó                          8082
124        Zaragoza                         12972

[125 rows x 2 columns]
